In [1]:
import re #Regular expression
import requests
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

In [2]:
## Step 1 : Download Html page 
response = requests.get('https://www.ted.com/talks?sort=newest&language=en')
response.status_code

200

In [3]:
html = response.text

In [4]:
item_lst=[]

In [5]:
driver = webdriver.Chrome(executable_path='chromedriver/chromedriver') # open browser 

### 1- Collect info ( speaker name, title, posted date and duration )

In [6]:
# to extract data from 60 pages
for i in range (1,61):
    next_page="https://www.ted.com/talks?language=en&page={}&sort=newest".format(i)
    
    driver.get(next_page)

    sleep(5) # sleep for few seconds to make sure that the initial page is opened
    html = driver.page_source
    soup = BeautifulSoup(html,'html')
    item_lst.append(soup.find_all('div',class_="media media--sm-v"))

In [7]:
second_lst=[]
#create nested loop to access nested list and append every item in the first list
for items1 in item_lst:
    for item in items1:
        second_lst.append(item)

In [8]:
# initiate the features
speaker_name=[]
title=[]
posted=[]
urls=[]
duration=[]

In [9]:
# bring the data to the lists
for i in second_lst:
    speaker_name.append(i.find('h4',class_="h12 talk-link__speaker").text.strip())
    title.append(i.find('h4',class_="f-w:700 h9 m5").text.strip())
    posted.append(i.find('span', class_="meta__val").text.strip())
    urls.append(i.find('a',class_="ga-link")['href'])
    duration.append(i.find('span', class_="thumb__duration").text.strip())

In [10]:
# make a dataframe
df = pd.DataFrame({'speaker_name':speaker_name,
                  'title':title,
                  'posted_date':posted,
                  'duration':duration,
                  'Link':urls})

In [11]:
# correct the links
df['Link']=df['Link'].apply(lambda x: 'https://www.ted.com'+x)

# print the dataframe
df

,speaker_name,title,posted_date,duration,Link
0,Alex Gendler,The Egyptian myth of the death of Osiris,Jul 2020,3:56,https://www.ted.com/talks/alex_gendler_the_egy...
1,Shari Davis,What if you could help decide how the governme...,Jul 2020,10:28,https://www.ted.com/talks/shari_davis_what_if_...
2,Nita Mosby Tyler,Want a more just world? Be an unlikely ally,Jul 2020,10:15,https://www.ted.com/talks/nita_mosby_tyler_wan...
3,Susan Lupack,The race to decode a mysterious language,Jul 2020,4:24,https://www.ted.com/talks/susan_lupack_the_rac...
4,Ariel Waldman,The colorful critter world of microbes in Anta...,Jul 2020,5:56,https://www.ted.com/talks/ariel_waldman_the_co...
...,...,...,...,...,...
2155,Nancy Frates,Meet the mom who started the Ice Bucket Challenge,Nov 2014,18:53,https://www.ted.com/talks/nancy_frates_meet_th...
2156,Will Marshall,Tiny satellites show us the Earth as it change...,Nov 2014,8:01,https://www.ted.com/talks/will_marshall_tiny_s...
2157,David Grady,How to save the world (or at least yourself) f...,Nov 2014,6:34,https://www.ted.com/talks/david_grady_how_to_s...
2158,Vincent Moon and Naná Vasconcelos,Hidden music rituals around the world,Nov 2014,24:13,https://www.ted.com/talks/vincent_moon_and_nan...


In [12]:
# save dataframe as csv file
df.to_csv(r'C:\Users\aafat\Desktop\DSI7\week7\project-3/ted_talks_v1.csv', index=False)

### 2- Collect info ( about speaker, about talk, views )

In [13]:
html = response.text

In [14]:
driver_vid = webdriver.Chrome(executable_path='chromedriver/chromedriver')

In [15]:
# initiate the lists to save the collecting data
talk_lst=[]
speaker_lst=[]
view_lst=[]

In [16]:
# to extract data from each video
for i in range (len(df['Link'])):
    next_video= df.Link[i]
    
    driver_vid.get(next_video)

    sleep(6) # sleep for few seconds to make sure that the initial page is opened
    html = driver_vid.page_source
    soup = BeautifulSoup(html,'html')
# we use try and except to avoid any error in this loop    
    try:
        test = soup.find('p' ,class_="l-h:n m-b:1").text
        talk_lst.append(test)
    except:
        talk_lst.append(soup.find('p' ,class_="l-h:n m-b:1"))

    try:
        test1 = soup.find('p', class_="d:n d:b@md l-h:n m-b:.5").text
        speaker_lst.append(test1)
    except:
        speaker_lst.append(soup.find('p', class_="d:n d:b@md l-h:n m-b:.5"))
    
    try:
        test2 = soup.find(attrs={'class':'css-1uodv95'}).text
        view_lst.append(test2)
    except:
        view_lst.append(soup.find(attrs={'class':'css-1uodv95'}))

In [17]:
# check the length of our lists
len(view_lst) 

2160

In [18]:
# initiate the features
about_speaker1=[]
about_talk1=[]
views1=[]

In [19]:
# bring the data to the lists
for i in talk_lst:
     about_talk1.append(i)

In [20]:
for s in speaker_lst:
     about_speaker1.append(s)

In [21]:
for v in view_lst:
    views1.append(v)

In [22]:
# make a dataframe
df2 = pd.DataFrame({'about_speaker':about_speaker1,
                  'about_talk':about_talk1,
                  'views':views1})

In [23]:
df2

,about_speaker,about_talk,views
0,,"Long jealous of his older brother Osiris, the ...","208,703"
1,As a leader of the Participatory Budgeting Pro...,What if you could help decide how the governme...,"425,688"
2,Nita Mosby Tyler specializes in the developmen...,A more equal world starts with you. Citing a f...,"460,269"
3,,"In the early 1900s, archaeologist Sir Arthur E...","350,202"
4,"An artist who's pivoted to science, Ariel Wald...","In this tour of the microscopic world, explore...","333,482"
...,...,...,...
2155,Nancy Frates and her family have raised a proj...,Remember the Ice Bucket Challenge craze this s...,"1,081,901"
2156,"At Planet, Will Marshall leads overall strateg...",Satellite imaging has revolutionized our knowl...,"1,914,256"
2157,David Grady is on a crusade to help you take b...,"An epidemic of bad, inefficient, overcrowded m...","2,504,886"
2158,"A jazz icon since the late 1960s, Naná Vasconc...",Vincent Moon travels the world with a backpack...,"1,104,616"


In [24]:
# save the 2nd dataframe as csv file
df2.to_csv(r'C:\Users\aafat\Desktop\DSI7\week7\project-3/ted_talks_more_details.csv', index=False)

In [25]:
# merge our dataframes
ted_original = pd.merge(df, df2, how='outer', right_index=True, left_index=True)
ted_original

,speaker_name,title,posted_date,duration,Link,about_speaker,about_talk,views
0,Alex Gendler,The Egyptian myth of the death of Osiris,Jul 2020,3:56,https://www.ted.com/talks/alex_gendler_the_egy...,,"Long jealous of his older brother Osiris, the ...","208,703"
1,Shari Davis,What if you could help decide how the governme...,Jul 2020,10:28,https://www.ted.com/talks/shari_davis_what_if_...,As a leader of the Participatory Budgeting Pro...,What if you could help decide how the governme...,"425,688"
2,Nita Mosby Tyler,Want a more just world? Be an unlikely ally,Jul 2020,10:15,https://www.ted.com/talks/nita_mosby_tyler_wan...,Nita Mosby Tyler specializes in the developmen...,A more equal world starts with you. Citing a f...,"460,269"
3,Susan Lupack,The race to decode a mysterious language,Jul 2020,4:24,https://www.ted.com/talks/susan_lupack_the_rac...,,"In the early 1900s, archaeologist Sir Arthur E...","350,202"
4,Ariel Waldman,The colorful critter world of microbes in Anta...,Jul 2020,5:56,https://www.ted.com/talks/ariel_waldman_the_co...,"An artist who's pivoted to science, Ariel Wald...","In this tour of the microscopic world, explore...","333,482"
...,...,...,...,...,...,...,...,...
2155,Nancy Frates,Meet the mom who started the Ice Bucket Challenge,Nov 2014,18:53,https://www.ted.com/talks/nancy_frates_meet_th...,Nancy Frates and her family have raised a proj...,Remember the Ice Bucket Challenge craze this s...,"1,081,901"
2156,Will Marshall,Tiny satellites show us the Earth as it change...,Nov 2014,8:01,https://www.ted.com/talks/will_marshall_tiny_s...,"At Planet, Will Marshall leads overall strateg...",Satellite imaging has revolutionized our knowl...,"1,914,256"
2157,David Grady,How to save the world (or at least yourself) f...,Nov 2014,6:34,https://www.ted.com/talks/david_grady_how_to_s...,David Grady is on a crusade to help you take b...,"An epidemic of bad, inefficient, overcrowded m...","2,504,886"
2158,Vincent Moon and Naná Vasconcelos,Hidden music rituals around the world,Nov 2014,24:13,https://www.ted.com/talks/vincent_moon_and_nan...,"A jazz icon since the late 1960s, Naná Vasconc...",Vincent Moon travels the world with a backpack...,"1,104,616"


In [26]:
# saving dataframe after merge
ted_original.to_csv(r'C:\Users\aafat\Desktop\DSI7\week7\project-3/ted_talks_v1_1.csv', index=False)

In [27]:
# read the merge dataset
sd = pd.read_csv("ted_talks_v1_1.csv")
sd

,speaker_name,title,posted_date,duration,Link,about_speaker,about_talk,views
0,Alex Gendler,The Egyptian myth of the death of Osiris,Jul 2020,3:56,https://www.ted.com/talks/alex_gendler_the_egy...,NaN,"Long jealous of his older brother Osiris, the ...","208,703"
1,Shari Davis,What if you could help decide how the governme...,Jul 2020,10:28,https://www.ted.com/talks/shari_davis_what_if_...,As a leader of the Participatory Budgeting Pro...,What if you could help decide how the governme...,"425,688"
2,Nita Mosby Tyler,Want a more just world? Be an unlikely ally,Jul 2020,10:15,https://www.ted.com/talks/nita_mosby_tyler_wan...,Nita Mosby Tyler specializes in the developmen...,A more equal world starts with you. Citing a f...,"460,269"
3,Susan Lupack,The race to decode a mysterious language,Jul 2020,4:24,https://www.ted.com/talks/susan_lupack_the_rac...,NaN,"In the early 1900s, archaeologist Sir Arthur E...","350,202"
4,Ariel Waldman,The colorful critter world of microbes in Anta...,Jul 2020,5:56,https://www.ted.com/talks/ariel_waldman_the_co...,"An artist who's pivoted to science, Ariel Wald...","In this tour of the microscopic world, explore...","333,482"
...,...,...,...,...,...,...,...,...
2155,Nancy Frates,Meet the mom who started the Ice Bucket Challenge,Nov 2014,18:53,https://www.ted.com/talks/nancy_frates_meet_th...,Nancy Frates and her family have raised a proj...,Remember the Ice Bucket Challenge craze this s...,"1,081,901"
2156,Will Marshall,Tiny satellites show us the Earth as it change...,Nov 2014,8:01,https://www.ted.com/talks/will_marshall_tiny_s...,"At Planet, Will Marshall leads overall strateg...",Satellite imaging has revolutionized our knowl...,"1,914,256"
2157,David Grady,How to save the world (or at least yourself) f...,Nov 2014,6:34,https://www.ted.com/talks/david_grady_how_to_s...,David Grady is on a crusade to help you take b...,"An epidemic of bad, inefficient, overcrowded m...","2,504,886"
2158,Vincent Moon and Naná Vasconcelos,Hidden music rituals around the world,Nov 2014,24:13,https://www.ted.com/talks/vincent_moon_and_nan...,"A jazz icon since the late 1960s, Naná Vasconc...",Vincent Moon travels the world with a backpack...,"1,104,616"


### 3- Collect info ( tags )

In [28]:
html = response.text

In [29]:
driver_vid = webdriver.Chrome(executable_path='chromedriver/chromedriver')

In [30]:
# initiate the lists to save the collecting data
tags_lst=[]

In [31]:
# to extract data from each video
for i in range (len(sd['Link'])):
    next_video= sd.Link[i]
    
    driver_vid.get(next_video)

    sleep(4) # sleep for few seconds to make sure that the initial page is opened
    html = driver_vid.page_source
    soup = BeautifulSoup(html,'html')
    
# we use try and except to avoid any error in this loop  
    try:
        test = soup.find_all('li', class_="t-t:c").text
        tags_lst.append(test)
    except:
        tags_lst.append(soup.find_all('li', class_="t-t:c"))

In [32]:
# check the length of our lists
len(tags_lst) 

2160

In [33]:
tags_lst

[[<li class="t-t:c"><a href="https://www.ted.com/topics/education">education</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/topics/ancient+world">ancient world</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/topics/ted-ed">TED-Ed</a></li>],
 [<li class="t-t:c"><a href="https://www.ted.com/topics/democracy">democracy</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/topics/leadership">leadership</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/topics/community">community</a></li>],
 [<li class="t-t:c"><a href="https://www.ted.com/topics/activism">activism</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/topics/inequality">inequality</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/topics/race">race</a></li>],
 [<li class="t-t:c"><a href="https://www.ted.com/topics/ted-ed">TED-Ed</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/topics/education">education</a></li>,
  <li class="t-t:c"><a href="https://www.ted.com/top

In [36]:
# make a dataframe
df3 = pd.DataFrame({'tags':tags_lst})

In [37]:
len(df3.tags)

2160

In [38]:
df3

,tags
0,"[[[education]], [[ancient world]], [[TED-Ed]]]"
1,"[[[democracy]], [[leadership]], [[community]]]"
2,"[[[activism]], [[inequality]], [[race]]]"
3,"[[[TED-Ed]], [[education]], [[language]]]"
4,"[[[science]], [[animals]], [[exploration]]]"
...,...
2155,"[[[TEDx]], [[disease]], [[family]]]"
2156,"[[[astronomy]], [[space]], [[technology]]]"
2157,"[[[business]], [[communication]], [[productivi..."
2158,"[[[film]], [[live music]], [[music]]]"


### Cleaning the tags column

In [39]:
# saving dataframe after merge
df3.to_csv(r'C:\Users\aafat\Desktop\DSI7\week7\project-3/tags.csv', index=False)

In [40]:
# read the merge dataset
tg = pd.read_csv("tags.csv")
tg

,tags
0,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
1,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
2,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
3,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
4,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
...,...
2155,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
2156,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
2157,"[<li class=""t-t:c""><a href=""https://www.ted.co..."
2158,"[<li class=""t-t:c""><a href=""https://www.ted.co..."


In [41]:
tg.isnull().sum()

tags    0
dtype: int64

In [42]:
# split value into 3 values
tg['tags'] = tg['tags'].apply(lambda x:x.split(','))

In [43]:
# clean any bad characters and print all tags after 2nd split
s = tg['tags']
w=[]
for i in s:
    for a in i:
        w.append(a.split('>')[2].split('<')[0])

In [44]:
len(w)

6459

In [45]:
w

['education',
 'ancient world',
 'TED-Ed',
 'democracy',
 'leadership',
 'community',
 'activism',
 'inequality',
 'race',
 'TED-Ed',
 'education',
 'language',
 'science',
 'animals',
 'exploration',
 'violence',
 'United States',
 'crime',
 'TED-Ed',
 'education',
 'animation',
 'science',
 'biology',
 'nature',
 'education',
 'society',
 'future',
 'TED-Ed',
 'animation',
 'government',
 'nature',
 'animals',
 'humanity',
 'technology',
 'robots',
 'innovation',
 'TED-Ed',
 'animation',
 'education',
 'math',
 'education',
 'history',
 'Africa',
 'journalism',
 'humor',
 'TED-Ed',
 'animation',
 'education',
 'TED-Ed',
 'education',
 'math',
 'education',
 'TED-Ed',
 'language',
 'gender',
 'race',
 'Gender spectrum',
 'art',
 'history',
 'race',
 'coronavirus',
 'pandemic',
 'science',
 'cancer',
 'illness',
 'life',
 'history',
 'food',
 'indigenous peoples',
 'life',
 'adventure',
 'personal growth',
 'leadership',
 'government',
 'women',
 'humanity',
 'community',
 'communicati

In [46]:
# combine 3 tags together
comp_lst= [w[x:x+3] for x in range(0, len(w),3)]

In [47]:
comp_lst

[['education', 'ancient world', 'TED-Ed'],
 ['democracy', 'leadership', 'community'],
 ['activism', 'inequality', 'race'],
 ['TED-Ed', 'education', 'language'],
 ['science', 'animals', 'exploration'],
 ['violence', 'United States', 'crime'],
 ['TED-Ed', 'education', 'animation'],
 ['science', 'biology', 'nature'],
 ['education', 'society', 'future'],
 ['TED-Ed', 'animation', 'government'],
 ['nature', 'animals', 'humanity'],
 ['technology', 'robots', 'innovation'],
 ['TED-Ed', 'animation', 'education'],
 ['math', 'education', 'history'],
 ['Africa', 'journalism', 'humor'],
 ['TED-Ed', 'animation', 'education'],
 ['TED-Ed', 'education', 'math'],
 ['education', 'TED-Ed', 'language'],
 ['gender', 'race', 'Gender spectrum'],
 ['art', 'history', 'race'],
 ['coronavirus', 'pandemic', 'science'],
 ['cancer', 'illness', 'life'],
 ['history', 'food', 'indigenous peoples'],
 ['life', 'adventure', 'personal growth'],
 ['leadership', 'government', 'women'],
 ['humanity', 'community', 'communicatio

In [48]:
len(comp_lst)

2153

In [49]:
# make this list as dataframe
tgs=pd.DataFrame(comp_lst,columns=['tag1','tag2','tag3'], dtype=str)

In [50]:
tgs

,tag1,tag2,tag3
0,education,ancient world,TED-Ed
1,democracy,leadership,community
2,activism,inequality,race
3,TED-Ed,education,language
4,science,animals,exploration
...,...,...,...
2148,invention,medicine,TEDx
2149,disease,family,astronomy
2150,space,technology,business
2151,communication,productivity,film


In [51]:
tgs.fillna({'tag3':'None'})

,tag1,tag2,tag3
0,education,ancient world,TED-Ed
1,democracy,leadership,community
2,activism,inequality,race
3,TED-Ed,education,language
4,science,animals,exploration
...,...,...,...
2148,invention,medicine,TEDx
2149,disease,family,astronomy
2150,space,technology,business
2151,communication,productivity,film


In [52]:
# combine 3 columns together
tgs["tags"] = tgs["tag1"] +','+ tgs["tag2"] +','+ tgs["tag3"]

In [53]:
tgs.drop(['tag1', 'tag2', 'tag3'], axis=1, inplace=True)

In [54]:
tgs

,tags
0,"education,ancient world,TED-Ed"
1,"democracy,leadership,community"
2,"activism,inequality,race"
3,"TED-Ed,education,language"
4,"science,animals,exploration"
...,...
2148,"invention,medicine,TEDx"
2149,"disease,family,astronomy"
2150,"space,technology,business"
2151,"communication,productivity,film"


In [57]:
# merge the new dataframe with old one
ted_main = pd.merge(sd, tgs, how='outer', right_index=True, left_index=True)

In [58]:
ted_main

,speaker_name,title,posted_date,duration,Link,about_speaker,about_talk,views,tags
0,Alex Gendler,The Egyptian myth of the death of Osiris,Jul 2020,3:56,https://www.ted.com/talks/alex_gendler_the_egy...,NaN,"Long jealous of his older brother Osiris, the ...","208,703","education,ancient world,TED-Ed"
1,Shari Davis,What if you could help decide how the governme...,Jul 2020,10:28,https://www.ted.com/talks/shari_davis_what_if_...,As a leader of the Participatory Budgeting Pro...,What if you could help decide how the governme...,"425,688","democracy,leadership,community"
2,Nita Mosby Tyler,Want a more just world? Be an unlikely ally,Jul 2020,10:15,https://www.ted.com/talks/nita_mosby_tyler_wan...,Nita Mosby Tyler specializes in the developmen...,A more equal world starts with you. Citing a f...,"460,269","activism,inequality,race"
3,Susan Lupack,The race to decode a mysterious language,Jul 2020,4:24,https://www.ted.com/talks/susan_lupack_the_rac...,NaN,"In the early 1900s, archaeologist Sir Arthur E...","350,202","TED-Ed,education,language"
4,Ariel Waldman,The colorful critter world of microbes in Anta...,Jul 2020,5:56,https://www.ted.com/talks/ariel_waldman_the_co...,"An artist who's pivoted to science, Ariel Wald...","In this tour of the microscopic world, explore...","333,482","science,animals,exploration"
...,...,...,...,...,...,...,...,...,...
2155,Nancy Frates,Meet the mom who started the Ice Bucket Challenge,Nov 2014,18:53,https://www.ted.com/talks/nancy_frates_meet_th...,Nancy Frates and her family have raised a proj...,Remember the Ice Bucket Challenge craze this s...,"1,081,901",NaN
2156,Will Marshall,Tiny satellites show us the Earth as it change...,Nov 2014,8:01,https://www.ted.com/talks/will_marshall_tiny_s...,"At Planet, Will Marshall leads overall strateg...",Satellite imaging has revolutionized our knowl...,"1,914,256",NaN
2157,David Grady,How to save the world (or at least yourself) f...,Nov 2014,6:34,https://www.ted.com/talks/david_grady_how_to_s...,David Grady is on a crusade to help you take b...,"An epidemic of bad, inefficient, overcrowded m...","2,504,886",NaN
2158,Vincent Moon and Naná Vasconcelos,Hidden music rituals around the world,Nov 2014,24:13,https://www.ted.com/talks/vincent_moon_and_nan...,"A jazz icon since the late 1960s, Naná Vasconc...",Vincent Moon travels the world with a backpack...,"1,104,616",NaN


In [64]:
# save the final csv file
ted_main.to_csv(r'C:\Users\aafat\Desktop\DSI7\week7\project-3/ted_main_v2.csv', index=False)